In [97]:
from sklearn import datasets
import pandas as pd
import gensim
import numpy as np
import csv
from nltk.corpus import stopwords
from nltk import word_tokenize
import string
import tqdm
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import text,sequence
from keras.models import Sequential,Model
from keras.layers import TimeDistributed,Lambda,Input,LSTM,Merge
from keras.layers.merge import Concatenate,Add
from keras.layers.core import Dense,Dropout,Activation,Flatten
from keras.layers.embeddings import Embedding
from keras.optimizers import Adadelta
from keras import optimizers
from keras.utils import np_utils
from keras.layers.advanced_activations import PReLU

In [139]:
x = pd.read_csv('questions.csv',usecols=['question1','question2','is_duplicate'])


In [146]:
x = pd.read_csv('questions.csv',usecols=['question1','question2','is_duplicate'])
stop_words = stopwords.words('english')
m = []
#print(x)

for sentence in (x.question1.values + x.question2.values.astype(str)):
    word = sentence.lower()
    word = word_tokenize(word)
    tmp = []
    for x in word:
        if (x not in stop_words) and (x not in string.punctuation):        
            tmp.append(x)
    m.append(tmp)        

model = gensim.models.Word2Vec(m,min_count=1,size=300)
embedding_index = {}
for w in model.wv.vocab:
    embedding_index[w] = model[w]

#print(embedding_index.get('banglore'))
#print(embedding_index)
df = pd.DataFrame.from_dict(embedding_index,orient='index')
df.to_csv('test.csv')
#data = pd.read_csv('test.csv')
data=csv.reader(open('test.csv'))
next(data)
new_dict = {}
for row in data:
    new_dict[row[0]] = row[1:] 

#print(new_dict)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [147]:
tk = text.Tokenizer(num_words=200000)
max_len = 40
x = pd.read_csv('questions.csv',usecols=['question1','question2','is_duplicate'])
tk.fit_on_texts(list(x.question1.values) + list(x.question2.values.astype(str)))
word_index=tk.word_index
#print((word_index))
y = x.is_duplicate
#print(ytrain_enc)
x1 = tk.texts_to_sequences(x.question1.values)
x1 = sequence.pad_sequences(x1, maxlen=max_len)

x2 = tk.texts_to_sequences(x.question2.values.astype(str))
x2 = sequence.pad_sequences(x2, maxlen=max_len)
#print(embedding_index['lonely'])    
#print(x1)

embedding_matrix = np.zeros((len(word_index)+1,300))
for word,i in word_index.items():
    tmp = new_dict.get(word)  
    if tmp is not None:        
        embedding_matrix[i] = tmp
       




In [148]:
from sklearn.cross_validation import train_test_split    
x1=pd.DataFrame(x1)
x2 = pd.DataFrame(x2)
x3 = pd.concat([x1,x2],axis=1,keys=['q1','q2'])
a,b,c,d = train_test_split(x3,y,test_size=.2)
#print(a['q2'])

In [149]:
def exponent_neg_manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))
# The visible layer
max_seq_length = 40
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')
embedding_dim=300
embedding_layer = Embedding(len(word_index)+1, embedding_dim, weights=[embedding_matrix], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(1)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Merge(mode=lambda x: exponent_neg_manhattan_distance(x[0], x[1]), output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [150]:
optimizer = Adadelta(clipnorm=0.5)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

In [151]:
malstm_trained = malstm.fit([a['q1'],a['q2']], [c] , batch_size=10, nb_epoch=10,validation_data=([b['q1'], b['q2']], [d]))
                            

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 323480 samples, validate on 80871 samples
Epoch 1/10
323480/323480 [==============================] - 3771s 12ms/step - loss: 0.3546 - acc: 0.4955 - val_loss: 0.3452 - val_acc: 0.5081
Epoch 2/10
323480/323480 [==============================] - 3729s 12ms/step - loss: 0.3433 - acc: 0.5102 - val_loss: 0.3411 - val_acc: 0.5136
Epoch 3/10
323480/323480 [==============================] - 3738s 12ms/step - loss: 0.3405 - acc: 0.5128 - val_loss: 0.3406 - val_acc: 0.5142
Epoch 4/10
323480/323480 [==============================] - 3733s 12ms/step - loss: 0.3383 - acc: 0.5155 - val_loss: 0.3395 - val_acc: 0.5156
Epoch 5/10
323480/323480 [==============================] - 3712s 11ms/step - loss: 0.3364 - acc: 0.5184 - val_loss: 0.3365 - val_acc: 0.5170
Epoch 6/10
323480/323480 [==============================] - 3761s 12ms/step - loss: 0.3357 - acc: 0.5194 - val_loss: 0.3363 - val_acc: 0.5201
Epoch 7/10
323480/323480 [==============================] - 3750s 12ms/step - loss: 0.3350 - acc:

In [195]:
"""s1 = tk.texts_to_sequences([['step diamond'],['step hello banglore']])
s1 = sequence.pad_sequences(s1,maxlen=40)
s2 = tk.texts_to_sequences([['step diamond'],['banglore yo ']])
s2 = sequence.pad_sequences(s2,maxlen=40)

malstm.predict([s1,s2])
#loss, accuracy = new_model.evaluate([s1,s2], [1,1], verbose=0)
#print(new_model.output)
#print(loss,accuracy)
"""

array([[ 1.],
       [ 1.]], dtype=float32)

In [170]:
data = pd.read_csv('test.csv')

In [189]:
q1 =  []
q2=[]
for i in data.question1.head(1000).values:
    q1.append([i])
for i in data.question2.head(1000).values:
    q2.append([i])




In [191]:
s1 = tk.texts_to_sequences(q1)
s1 = sequence.pad_sequences(s1,maxlen=40)
s2 = tk.texts_to_sequences(q2)
s2 = sequence.pad_sequences(s2,maxlen=40)

In [193]:

pred = malstm.predict([s1,s2])
#best_pred = np.array([np.argmax(line) for line in pred])
print(pred)

[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]